In [254]:
import numpy as np
import pandas as pd
import math

In [296]:
# 1A
def read(filename):
    data_raw = pd.read_csv(filename)
    print ('The total number of rows for original dataframe are:', data_raw.iloc[:,0].size)
    print ('The total number of columns for original dataframe are:', data_raw.columns.size)
    return data_raw

In [308]:
def randomSp(data_raw, sd):
    arr = data_raw.values.reshape(data_raw.iloc[:,0].size, data_raw.columns.size)
    name, data = np.vsplit(arr,[1])
    np.random.seed(seed = sd)
    np.random.shuffle(data)
    n = math.ceil((data_raw.iloc[:,0].size/5))+1
    arr1,arr2,arr3,arr4,arr5 = np.split(data,[n,2*n,3*n,4*n])
    df1 = pd.DataFrame(arr1)
    df2 = pd.DataFrame(arr2)
    df3 = pd.DataFrame(arr3)
    df4 = pd.DataFrame(arr4)
    df5 = pd.DataFrame(arr5)
    df1.columns = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age','Class']
    df2.columns = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age','Class']
    df3.columns = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age','Class']
    df4.columns = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age','Class']
    df5.columns = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age','Class']
    frames = [df2, df3, df4, df5]
    train = pd.concat(frames, ignore_index=True)
    test = df1
    return [train,test]

In [309]:
def calculateFeature(dataSet):
    n_0 = dataSet['Class'][dataSet['Class'] == 0.0].count()
    n_1 = dataSet['Class'][dataSet['Class'] == 1.0].count()
    total = dataSet['Class'].count()
    P_0 = n_0/total
    P_1 = n_1/total
    data_means = dataSet.groupby('Class').mean()
    data_variance = dataSet.groupby('Class').var()
    index = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age']
    A_mean = {}
    A_variance = {}
    B_mean = {}
    B_variance = {}
    for i in range(8):
        A_mean[i] = data_means[index[i]][data_means.index == 0.0].values[0]
        A_variance[i] = data_variance[index[i]][data_variance.index == 0.0].values[0]
        B_mean[i] = data_means[index[i]][data_means.index == 1.0].values[0]
        B_variance[i] = data_variance[index[i]][data_variance.index == 1.0].values[0]
    return [P_0, P_1, A_mean, A_variance, B_mean, B_variance]

In [327]:
def calculateProbability(x, mean_y, variance_y):
    p = 1/(math.sqrt(2*math.pi*variance_y)) * math.exp((-(x-mean_y)**2)/(2*variance_y))
    return p

In [328]:
def predict(A_mean, A_variance, B_mean, B_variance, P_0, P_1, test):
    index = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age']
    Class = {}
    p_0 = {}
    p_1 = {}
    for j in range(test.iloc[:,0].size):
        p_0[j] = P_0
        p_1[j] = P_1
        for i in range(8):
            #p_0[j] *= calculatePro(test[index[i]].values[j], A_mean[i], A_variance[i])
            #p_1[j] *= calculatePro(test[index[i]].values[j], B_mean[i], B_variance[i])
            p_0[j] *= calculateProbability(test[index[i]].values[j], A_mean[i], A_variance[i])
            p_1[j] *= calculateProbability(test[index[i]].values[j], B_mean[i], B_variance[i])
        if p_0[j] >= p_1[j]:
            Class[j] = 0.0
        else: 
            Class[j] = 1.0
    return Class

In [329]:
def prediction(Class,test):
    true = 0
    for i in range(len(Class)):
        if Class[i] == test['Class'][i]:
            true += 1
    return (true/float(len(Class)))*100.0

In [377]:
def mainA():
    filename = 'diabetes.csv'
    data_raw = read(filename)
    Acc = {}
    for sd in range(10):
        train,test = randomSp(data_raw, sd)
        P_0, P_1, A_mean, A_variance, B_mean, B_variance = calculateFeature(train)
        Class = predict(A_mean, A_variance, B_mean, B_variance, P_0, P_1, test)
        Acc[sd] = prediction(Class,test)
    print('The accuarcies for 10 times is:',Acc)
    m = 0
    for i in range(len(Acc)):
        m += Acc[i]
    Av_Acc = m/len(Acc)
    print('The mean accuarcy is:',Av_Acc)    

In [378]:
mainA()

The total number of rows for original dataframe are: 768
The total number of columns for original dataframe are: 9
The accuarcies for 10 times is: {0: 80.64516129032258, 1: 73.54838709677419, 2: 78.06451612903226, 3: 73.54838709677419, 4: 76.12903225806451, 5: 76.12903225806451, 6: 75.48387096774194, 7: 78.06451612903226, 8: 77.41935483870968, 9: 72.90322580645162}
The mean accuarcy is: 76.19354838709675


In [346]:
#1B
def calculateFeature2(dataSet):
    n_0 = dataSet['Class'][dataSet['Class'] == 0.0].count()
    n_1 = dataSet['Class'][dataSet['Class'] == 1.0].count()
    total = dataSet['Class'].count()
    P_0 = n_0/total
    P_1 = n_1/total
    index = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age']
    A_variance = {}
    B_mean = {}
    B_variance = {}
    count_0 = [0,0,0,0,0,0,0,0]
    tal_0 = [0,0,0,0,0,0,0,0]
    count_1 = [0,0,0,0,0,0,0,0]
    tal_1 = [0,0,0,0,0,0,0,0]
    var_0 = [0,0,0,0,0,0,0,0]
    con_0 = [0,0,0,0,0,0,0,0]
    var_1 = [0,0,0,0,0,0,0,0]
    con_1 = [0,0,0,0,0,0,0,0]
    for i in [2,3,5,7]:
        for m in range(dataSet.iloc[:,0].size):
            if dataSet['Class'][m] == 0.0:
                if dataSet[index[i]][m] == 0.0:
                    count_0[i] = count_0[i] 
                    tal_0[i] = tal_0[i] + dataSet[index[i]][m]
                else:
                    count_0[i] = count_0[i] + 1
                    tal_0[i] = tal_0[i] + dataSet[index[i]][m]
            else:
                if dataSet[index[i]][m] == 0.0:
                    count_1[i] = count_1[i] 
                    tal_1[i] = tal_1[i] + dataSet[index[i]][m]
                else:
                    count_1[i] = count_1[i] + 1
                    tal_1[i] = tal_1[i] + dataSet[index[i]][m]
        A_mean[i] = tal_0[i]/count_0[i]
        B_mean[i] = tal_1[i]/count_1[i]
    for i in [2,3,5,7]:
        for m in range(dataSet.iloc[:,0].size):
            if dataSet['Class'][m] == 0.0:
                if dataSet[index[i]][m] == 0.0:
                    con_0[i] = con_0[i] 
                    var_0[i] = var_0[i]
                else:
                    con_0[i] = con_0[i] + 1
                    var_0[i] = var_0[i] + math.pow((dataSet[index[i]][m]-A_mean[i]),2)
            else:
                if dataSet[index[i]][m] == 0.0:
                    con_1[i] = con_1[i] 
                    var_1[i] = var_1[i]
                else:
                    con_1[i] = con_1[i] + 1
                    var_1[i] = var_1[i] + math.pow((dataSet[index[i]][m]-A_mean[i]),2)
        A_variance[i] = var_0[i]/con_0[i]
        B_variance[i] = var_1[i]/con_1[i]
    data_means = dataSet.groupby('Class').mean()
    data_variance = dataSet.groupby('Class').var()
    for i in [0,1,4,6]:
        A_mean[i] = data_means[index[i]][data_means.index == 0.0].values[0]
        A_variance[i] = data_variance[index[i]][data_variance.index == 0.0].values[0]
        B_mean[i] = data_means[index[i]][data_means.index == 1.0].values[0]
        B_variance[i] = data_variance[index[i]][data_variance.index == 1.0].values[0]
    return [P_0, P_1, A_mean, A_variance, B_mean, B_variance]

In [371]:
def predict2(A_mean, A_variance, B_mean, B_variance, P_0, P_1, test):
    index = ['Pregancies', 'Glucose','BloodPressure','SkinThickness','Insulin','BMI','DPF','Age']
    Class = {}
    p_0 = {}
    p_1 = {}
    for j in range(test.iloc[:,0].size):
        p_0[j] = P_0
        p_1[j] = P_1
        for i in [2,3,5,7]:
            if test[index[i]].values[j] != 0.0:
            #p_0[j] *= calculatePro(test[index[i]].values[j], A_mean[i], A_variance[i])
            #p_1[j] *= calculatePro(test[index[i]].values[j], B_mean[i], B_variance[i])
                p_0[j] *= calculateProbability(test[index[i]].values[j], A_mean[i], A_variance[i])
                p_1[j] *= calculateProbability(test[index[i]].values[j], B_mean[i], B_variance[i])
        for i in [0,1,4,6]:
            p_0[j] *= calculateProbability(test[index[i]].values[j], A_mean[i], A_variance[i])
            p_1[j] *= calculateProbability(test[index[i]].values[j], B_mean[i], B_variance[i])
        if p_0[j] >= p_1[j]:
            Class[j] = 0.0
        else: 
            Class[j] = 1.0
    return Class

In [379]:
def mainB():
    filename = 'diabetes.csv'
    data_raw = read(filename)
    Acc = {}
    for sd in range(10):
        train,test = randomSp(data_raw, sd)
        P_0, P_1, A_mean, A_variance, B_mean, B_variance = calculateFeature2(train)
        Class = predict2(A_mean, A_variance, B_mean, B_variance, P_0, P_1, test)
        Acc[sd] = prediction(Class,test)
    print('The accuarcies for 10 times is:',Acc)
    m = 0
    for i in range(len(Acc)):
        m += Acc[i]
    Av_Acc = m/len(Acc)
    print('The mean accuarcy is:',Av_Acc)    

In [380]:
mainB()

The total number of rows for original dataframe are: 768
The total number of columns for original dataframe are: 9
The accuarcies for 10 times is: {0: 78.70967741935485, 1: 70.3225806451613, 2: 74.83870967741936, 3: 70.96774193548387, 4: 72.90322580645162, 5: 75.48387096774194, 6: 69.6774193548387, 7: 76.12903225806451, 8: 78.06451612903226, 9: 71.61290322580646}
The mean accuarcy is: 73.87096774193547
